# Formation PyTorch : les bases pour être autonome 
#### 3 novembre 2022 de 9h à 17h à l'OMP (salle Coriolis)

# Partie 4
## Modélisation probabiliste avec PyTorch

Dans cette dernière partie, nous allons nous initier à la modélisation probabiliste avec PyTorch avec un exemple de régression linéaire bayésienne. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt

In [ ]:
X = torch.linspace(0, 10, 100).unsqueeze(1)
Y = 2.5 + 3*X + 3*torch.randn((100,1))

fig = plt.figure()
plt.scatter(X, Y)
plt.show()

Plutôt que de calculer le maximum de vraisemblance $w^\ast$ des paramètres du modèle linéaire (régression linéaire classique), l'objectif d'une régression linéaire bayésienne est de calculer la distribution des paramètres du modèle linéaire. On modélise la vraisemblance des données $p(y \vert x, w)$ et l'a priori sur les paramètres du modèle $p(w)$ ainsi :

$$p(y \vert x, w) = \mathcal{N}(y \vert x^T w_1 + w_0, \sigma)$$
$$p(w) = \mathcal{N}(w \vert \mu_\circ, \sigma_\circ)$$

Dans le cas présent, on pourrait utiliser la loi de Bayes pour calculer analytiquement la distribution a posteriori $p(w \vert x, y)$. Néanmoins, pour des problèmes non linéaires, il n'y a souvent pas de solutions analytiques. 

Ici, on propose d'implémenter avec PyTorch une approche variationnelle (https://en.wikipedia.org/wiki/Variational_Bayesian_methods) qui consiste à approximer la distribution a posteriori par une distribution $q_\phi(w)$ paramétrée par $\phi$. Dans notre cas, $\phi = [\mu(w_0), \sigma(w_0), \mu(w_1), \sigma(w_1)]$, les moyennes et les écarts-types de l'ordonnée à l'origine et du coefficient directeur.

La fonction objective est une borne supérieure de la log-vraisemblance négative -log $p(x,y)$ : 
$$\mathcal{L}(x,y) = - \mathbb{E}_{q_\phi(w)} \big[\mbox{log} \: p(y \vert x, w)\big] + D_{KL}\big(q_\phi(w) \vert\vert p(w)\big)$$

où $D_{KL}$ désigne la divergence de Kullback-Leibler (https://fr.wikipedia.org/wiki/Divergence_de_Kullback-Leibler), que l'on peut calculer avec PyTorch (https://pytorch.org/docs/stable/distributions.html?highlight=torch+distributions+kl#module-torch.distributions.kl).

Comme la vraisemblance est gaussienne, minimiser le terme - log $p(y \vert x, w)$ revient à minismer l'erreur au carré entre les vrais $y_i$ et les $\hat{y}_i$ prédits. 

Par ailleurs, on approxime l'espérance $\mathbb{E}_{q_\phi(w)}$ avec des échantillons de Monte Carlo comme suit : 
$$ \mathbb{E}_{q_\phi(w)} \big[\mbox{log} \: p(y \vert x, w)\big] \approx \frac{1}{K} \sum_k \mbox{log} \: p(y \vert x, w^k)$$
où $w^k \sim q_\phi(w)$. En pratique, on prend $K=1$.

Ainsi, le modèle linéaire Bayésien est stochastique. Pour efficacement rétro-propager des gradients à travers des couches stochastiques, on va utiliser le "reparametrization trick" introduit dans [1]. Au lieu d'échantillonner $w^k$ à partir de $q_\phi(w)$, on échantillonne un bruit $\epsilon^k$ indépendant de $\phi$:

$$\epsilon^k \sim \mathcal{N}(0,1)$$
$$w^k = \mu(w) + \epsilon^k \cdot \sigma(w)$$

[1] Kingma, D. P., & Welling, M. (2013). Auto-encoding variational bayes. arXiv preprint arXiv:1312.6114.

In [ ]:
class BayesianLinearModel(torch.nn.Module):
    def __init__(self, x_dim):
        super(BayesianLinearModel, self).__init__()
        self.x_dim = x_dim
        # A compléter : définir q_\phi(w)
        ...
        
    def IC(self, level=0.95):
        """
        Calcule l'intervalle de confiance de niveau 'level'
        pour les paramètres du modèle.
        """
        # A compléter 
        ...
        return low, high
        
    def forward(self, x):
        # A compléter : utiliser le "reparametrization trick"
        ... 
        return out

In [ ]:
model = BayesianLinearModel(X.shape[1])
prior = torch.distributions.normal.Normal(torch.zeros(2,1), 4*torch.ones(2,1))
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
beta = 1e-3

for epoch in range(20):
    y_pred = model(X)
    posterior = ...
    kld = torch.distributions.kl.kl_divergence(posterior, prior).sum()
    mse = F.mse_loss(Y, y_pred)
    loss = mse + beta*kld
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f'=== Epoch {epoch} ===  ')
    print(f'MSE: {mse.item():.3f}')

In [ ]:
low_w, high_w = model.IP()

with torch.no_grad():
    # A compléter
    y_mean = ...
    y_low = ...
    y_high = ...
    
fig = plt.figure()
plt.scatter(X, Y)
plt.plot(X, y_pred, color='red')
plt.fill_between(X.view(-1), y_low.view(-1), y_high.view(-1), alpha=0.3)
plt.show()